In [ ]:
# regime_inference.py
import numpy as np
import pandas as pd

def _ols_slope(y: np.ndarray) -> float:
    t = np.arange(len(y))
    X = np.vstack([t, np.ones_like(t)]).T
    m, _ = np.linalg.lstsq(X, y, rcond=None)[0]
    return m


def _slope_vol_reg(close: np.ndarray,
                   idx:   int,
                   slope_win: int = 30,
                   vol_win:   int = 100
                  ) -> float | int:
    logp = np.log(close)

    # 1) slope
    slope_series = (
        pd.Series(logp)
          .rolling(slope_win, min_periods=slope_win)
          .apply(_ols_slope, raw=True)
    )
    rtn        = pd.Series(logp).diff()
    vol_series = rtn.rolling(vol_win, min_periods=1).std()

    slope = slope_series.iloc[idx]
    vol   = vol_series.iloc[idx]
    if np.isnan(slope) or np.isnan(vol):
        return np.nan

    # 3) causal median of vol_series up to idx
    median_vol = vol_series.iloc[: idx + 1].median()

    return 2 if (slope > 0 and vol < median_vol) else 0



def compute_regime_features_window(prices_window: np.ndarray) -> np.ndarray:

    n_inst, win_len = prices_window.shape
    idx = win_len - 1                   

    out = np.full((n_inst, 9), np.nan)
    sqrt_weights = np.arange(1, 46, dtype=float) ** 0.5
    sqrt_weights /= sqrt_weights.sum()

    for i in range(n_inst):
        close = prices_window[i]
        logp  = np.log(close)

        # MA regime
        ma_s = pd.Series(logp).rolling(5).mean().iloc[idx]
        ma_l = pd.Series(logp).rolling(70).mean().iloc[idx]
        ma_reg = 0 if ma_l > ma_s else 2

        # EMA regime
        ema_s = pd.Series(logp).ewm(span=5,  adjust=False).mean().iloc[idx]
        ema_l = pd.Series(logp).ewm(span=50, adjust=False).mean().iloc[idx]
        ema_reg = 2 if ema_s > ema_l else 0

        # Slope/Vol regime
        sv_reg = _slope_vol_reg(close, idx)

        # MACD regime
        macd_line = (
            pd.Series(logp).ewm(50, adjust=False).mean()
            - pd.Series(logp).ewm(90, adjust=False).mean()
        )
        signal_line = macd_line.ewm(span=40, adjust=False).mean()
        macd_reg = 2 if macd_line.iloc[idx] > signal_line.iloc[idx] else 0

        # Kalman trend regime
        proc_var, meas_var = 0.01, 10.0
        x_est = np.zeros(win_len)
        P     = np.zeros(win_len)
        x_est[0], P[0] = logp[0], 1.0
        for t in range(1, win_len):
            x_pred = x_est[t - 1]
            P_pred = P[t - 1] + proc_var
            K      = P_pred / (P_pred + meas_var)
            x_est[t] = x_pred + K * (logp[t] - x_pred)
            P[t]     = (1 - K) * P_pred
        kalman_reg = 2 if logp[idx] > x_est[idx] else 0

        # Fibonacci regime
        if idx >= 50:
            win50 = close[idx - 49 : idx + 1]
            hi, lo = win50.max(), win50.min()
            rng = hi - lo
            upper, lower = lo + 0.786 * rng, lo + 0.618 * rng
            fib_reg = 2 if close[idx] > upper else 0 if close[idx] < lower else 1
        else:
            fib_reg = np.nan

        # PSAR regime
        psar = np.empty(win_len)
        trend_up, af, max_af = True, 0.01, 0.10
        ep = close[0]
        psar[0] = close[0]
        for t in range(1, win_len):
            psar[t] = psar[t - 1] + af * (ep - psar[t - 1])
            if trend_up:
                if close[t] < psar[t]:
                    trend_up, psar[t], ep, af = False, ep, close[t], 0.01
                elif close[t] > ep:
                    ep, af = close[t], min(af + 0.01, max_af)
            else:
                if close[t] > psar[t]:
                    trend_up, psar[t], ep, af = True, ep, close[t], 0.01
                elif close[t] < ep:
                    ep, af = close[t], min(af + 0.01, max_af)
        psar_reg = 2 if close[idx] > psar[idx] else 0

        # Z-score regime
        ma90 = pd.Series(close).rolling(90).mean().iloc[idx]
        sd90 = pd.Series(close).rolling(90).std().iloc[idx]
        if np.isnan(ma90) or np.isnan(sd90):
            zscore_reg = np.nan
        else:
            z = (close[idx] - ma90) / sd90
            zscore_reg = 2 if z > 0.5 else 0 if z < -0.5 else 1

        # Weighted-return regime
        if idx >= 45:
            r = pd.Series(close).pct_change().iloc[idx - 44 : idx + 1].values
            wr = np.dot(r, sqrt_weights)
            wret_reg = 2 if wr > 0 else 0 if wr < 0 else 1
        else:
            wret_reg = np.nan

        out[i] = [
            ma_reg, ema_reg, sv_reg, macd_reg, kalman_reg,
            fib_reg, psar_reg, zscore_reg, wret_reg,
        ]

    return out

def _extract_window(price_file: str,
                    timestep: int,
                    win_len: int = 100) -> np.ndarray:
    """
    Slice the latest `win_len` bars (inclusive) ending at `timestep` from the
    price file and transpose to (n_inst, win_len).
    """
    df = pd.read_csv(price_file, sep=r"\s+", header=None)
    n_rows, n_inst = df.shape

    if not (0 <= timestep < n_rows):
        raise ValueError(f"timestep {timestep} out of range (0 … {n_rows-1})")
    if timestep < win_len - 1:
        raise ValueError("Not enough history to build a 100-bar window.")

    slice_df = df.iloc[timestep - win_len + 1 : timestep + 1, :]
    return slice_df.to_numpy().T            # (n_inst, win_len)


def infer_from_file(price_file: str,
                    timestep: int) -> np.ndarray:
    """
    High-level convenience wrapper:
    1. read prices.txt
    2. build the (50,100) window ending at `timestep`
    3. run the regime-feature pipeline
    """
    window = _extract_window(price_file, timestep, win_len=100)
    #print(len(window[0]))
    return compute_regime_features_window(window)


In [ ]:
import numpy as np
import pandas as pd

from precision_labeller import plot_all_regimes_long

def build_feature_label_csv(price_file: str,
                            N: int = 740,
                            output_csv: str = "features_labels.csv"):
    """
    Runs through timesteps 0..N-1 of prices.txt, builds the 9-regime features
    for each instrument at each t, pulls in the true_autolabel, and saves
    a long-form CSV indexed by instrument->time.
    """
    # 1) load prices once
    df_price = pd.read_csv(price_file, sep=r"\s+", header=None)
    n_rows, n_inst = df_price.shape
    assert N <= n_rows, f"N={N} exceeds available rows={n_rows}"

    # 2) precompute true regimes for each instrument
    #    this returns an array length N for each inst
    true_regs = {
        inst: plot_all_regimes_long(end_point=N + 10, plot_graph=False, inst=inst)
        for inst in range(n_inst)
    }

    # 3) iterate timesteps and call infer_from_file
    records = []
    for t in range(N):
        try:
            # infer_from_file expects timestep index in [0..]
            feats_t = infer_from_file(price_file, timestep=t)
            # feats_t is shape (n_inst, 9)
        except ValueError:
            # not enough history (t < 99), fill with NaNs
            feats_t = np.full((n_inst, 9), np.nan)

        for inst in range(n_inst):
            row = {
                "inst": inst,
                "time": t,
                "ma":          feats_t[inst, 0],
                "ema":         feats_t[inst, 1],
                "slope_vol":   feats_t[inst, 2],
                "macd":        feats_t[inst, 3],
                "kalman":      feats_t[inst, 4],
                "fib":         feats_t[inst, 5],
                "psar":        feats_t[inst, 6],
                "zscore":      feats_t[inst, 7],
                "wret":        feats_t[inst, 8],
                "true_regime": true_regs[inst][t]
            }
            records.append(row)

    # 5) build DataFrame & save
    df = pd.DataFrame.from_records(records)
    df = df.sort_values(["inst", "time"]).reset_index(drop=True)
    df.to_csv(output_csv, index=False)
    print(f"Wrote {len(df)} rows to {output_csv}")

if __name__ == "__main__":
    build_feature_label_csv("prices.txt", N=740, output_csv="features_labels.csv")


In [ ]:

features_t451 = infer_from_file("prices.txt", timestep=99)
# features_t451.shape  ->  (50, 9)
print(features_t451)  # Example output for the first instrument


In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# ── 2) Hyperparameters
SEQ_LEN     = 20
BATCH_SIZE  = 64
HIDDEN_SIZE = 64
NUM_LAYERS  = 2
DROPOUT     = 0.2
LR          = 1e-3
NUM_EPOCHS  = 20
TEST_SIZE   = 0.3
RANDOM_SEED = 42

# ── 3) Dataset & Model
class RegimeDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).long()
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class RegimeLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)      # (batch, seq_len, hidden)
        out = out[:, -1, :]        # last time-step
        return self.fc(out)        # (batch, num_classes)

# ── 4) Load & preprocess data
df = pd.read_csv("features_all_models_FINAL.csv")  # features with true_regime
# drop first-100 warmups per instrument
df = (
    df
    .groupby("inst", group_keys=False)
    .apply(lambda g: g.iloc[100:])
    .reset_index(drop=True)
)
price_df = pd.read_csv("prices.txt", sep=r"\s+", header=None)

# build sequences for LSTM
X_raw = df.drop(["inst","time","true_regime"], axis=1).values
y_raw = df["true_regime"].values

X_seqs, y_seqs, inst_map = [], [], []
for inst in df["inst"].unique():
    mask = df["inst"] == inst
    Xi, yi = X_raw[mask], y_raw[mask]
    for t in range(SEQ_LEN, len(Xi)):
        X_seqs.append(Xi[t-SEQ_LEN : t])  # past SEQ_LEN steps
        y_seqs.append(yi[t])              # regime label at t
        inst_map.append(inst)

X_seqs   = np.stack(X_seqs)  # (N, SEQ_LEN, D)
y_seqs   = np.array(y_seqs)  # (N,)
inst_map = np.array(inst_map)

# ── 5) Train/val split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X_seqs, y_seqs,
    test_size=TEST_SIZE,
    stratify=y_seqs,
    random_state=RANDOM_SEED
)

train_loader = DataLoader(RegimeDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(RegimeDataset(X_val,   y_val),   batch_size=BATCH_SIZE)

# ── 6) Build & train the LSTM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RegimeLSTM(
    input_size  = X_seqs.shape[2],
    hidden_size = HIDDEN_SIZE,
    num_layers  = NUM_LAYERS,
    num_classes = int(y_raw.max())+1,
    dropout     = DROPOUT
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

for epoch in range(1, NUM_EPOCHS+1):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for Xb, yb in train_loader:
        Xb, yb = Xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model(Xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * Xb.size(0)
        preds = out.argmax(dim=1)
        correct += (preds == yb).sum().item()
        total   += yb.size(0)
    print(f"Epoch {epoch:02d}  Loss: {total_loss/total:.4f}  Acc: {correct/total:.4f}")

# ── 7) Inference & separate plotting per instrument
def get_segments(reg):
    changes = np.flatnonzero(reg[1:] != reg[:-1])
    starts  = np.concatenate(([0], changes+1))
    ends    = np.concatenate((changes, [len(reg)-1]))
    return list(zip(starts, ends, reg[starts]))

# Flip coloring: red for bullish, grey for neutral, green for bearish
true_cmap = ListedColormap(["#ffcccc", "#f0f0f0", "#ccffcc"])
pred_cmap = ListedColormap(["#ff6666", "#b0b0b0", "#66cc66"])

model.eval()
with torch.no_grad():
    for inst in sorted(np.unique(inst_map)):
        # assemble data
        mask      = inst_map == inst
        Xi        = torch.from_numpy(X_seqs[mask]).float().to(device)
        true_i    = y_seqs[mask]
        pred_i    = model(Xi).argmax(dim=1).cpu().numpy()
        # reconstruct full-length arrays
        Ni        = df[df["inst"] == inst].shape[0]
        price_i   = price_df.iloc[100:100+Ni, inst].values
        true_full = np.concatenate([np.full(SEQ_LEN, np.nan), true_i])
        pred_full = np.concatenate([np.full(SEQ_LEN, np.nan), pred_i])
        # plot true regimes
        fig, ax = plt.subplots(figsize=(12,4))
        for s,e,lbl in get_segments(true_full[~np.isnan(true_full)].astype(int)):
            ax.axvspan(s+SEQ_LEN, e+SEQ_LEN, color=true_cmap(lbl), alpha=0.3)
        ax.plot(price_i, color="k", label="Price")
        ax.set_title(f"Inst {inst} — TRUE regimes vs Price")
        ax.set_xlabel("Time Step")
        ax.set_ylabel("Price")
        ax.legend()
        plt.show()
        # plot predicted regimes
        fig, ax = plt.subplots(figsize=(12,4))
        for s,e,lbl in get_segments(pred_full[~np.isnan(pred_full)].astype(int)):
            ax.axvspan(s+SEQ_LEN, e+SEQ_LEN, color=pred_cmap(lbl), alpha=0.3)
        ax.plot(price_i, color="k", label="Price")
        ax.set_title(f"Inst {inst} — PREDICTED regimes vs Price")
        ax.set_xlabel("Time Step")
        ax.set_ylabel("Price")
        ax.legend()
        plt.show()


In [ ]:
import numpy  as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# ─── CONFIG ────────────────────────────────────────────────────────────────
FEATURE_CSV    = "features_labels.csv"  # must include inst,time,ma…wret,true_regime
PRICE_CSV      = "prices.txt"
PAST_LEN       = 20     # how many past bars
FUTURE_LEN     = 20     # how many future bars for look-ahead
TRAIN_START    = 100    # drop first 100 warm-ups
TRAIN_END      = 600    # train up to t=599
TEST_START     = 600    # test from t=600
TEST_END       = 1000    # …to t=749
BATCH_SIZE     = 64
HIDDEN_SIZE    = 64
NUM_LAYERS     = 2
DROPOUT        = 0.2
LR             = 1e-3
NUM_EPOCHS     = 10
DEVICE         = torch.device("cuda" if torch.cuda.is_available() else "cpu")
FEAT_COLS      = ["ma","ema","slope_vol","macd","kalman","fib","psar","zscore","wret"]

# ─── DATASETS ───────────────────────────────────────────────────────────────
class LookaheadDataset(Dataset):
    def __init__(self, df, start_t, end_t, past_len, future_len, feat_cols):
        self.X, self.Y = [], []
        self.meta = []  # store (inst, time)
        for inst, g in df.groupby("inst"):
            g = g.sort_values("time").set_index("time")
            feats = g[feat_cols].values
            labs  = g["true_regime"].values
            times = g.index.values
            for i, t in enumerate(times):
                if t < start_t or t >= end_t: continue
                if i < past_len or i+future_len >= len(times): continue
                past   = feats[i-past_len  : i    ]
                future = feats[i+1          : i+1+future_len]
                window = np.vstack([past, future])
                self.X.append(window)
                self.Y.append(labs[i])
                self.meta.append((inst, t))
        self.X = torch.from_numpy(np.stack(self.X)).float()
        self.Y = torch.from_numpy(np.array(self.Y)).long()

    def __len__(self): return len(self.Y)
    def __getitem__(self, i): return self.X[i], self.Y[i]

class PastSelfLabelDataset(Dataset):
    def __init__(self, past_windows, labels):
        self.X = torch.from_numpy(np.stack(past_windows)).float()
        self.Y = torch.from_numpy(np.array(labels)).long()

    def __len__(self): return len(self.Y)
    def __getitem__(self, i): return self.X[i], self.Y[i]

# ─── MODEL ─────────────────────────────────────────────────────────────────
class BiLSTM(nn.Module):
    def __init__(self, feat_dim, hidden, layers, n_labels, dropout):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size   = feat_dim,
            hidden_size  = hidden,
            num_layers   = layers,
            batch_first  = True,
            dropout      = dropout,
            bidirectional= True
        )
        self.fc = nn.Linear(hidden*2, n_labels)

    def forward(self, x):
        out, _ = self.lstm(x)
        out    = out[:, -1, :]
        return self.fc(out)

# ─── 1) LOAD & INITIAL TRAIN/INFER ──────────────────────────────────────────
df = pd.read_csv(FEATURE_CSV)
df = df[df.time >= TRAIN_START].reset_index(drop=True)
n_labels = int(df.true_regime.max())+1
feat_dim = len(FEAT_COLS)

# initial train
train_ds = LookaheadDataset(df, TRAIN_START, TRAIN_END, PAST_LEN, FUTURE_LEN, FEAT_COLS)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
model1 = BiLSTM(feat_dim, HIDDEN_SIZE, NUM_LAYERS, n_labels, DROPOUT).to(DEVICE)
opt1  = torch.optim.Adam(model1.parameters(), lr=LR)
crit  = nn.CrossEntropyLoss()
for epoch in range(1, NUM_EPOCHS+1):
    model1.train(); total_loss=0; total_n=0
    for Xb, yb in train_loader:
        Xb, yb = Xb.to(DEVICE), yb.to(DEVICE)
        logits = model1(Xb); loss = crit(logits, yb)
        opt1.zero_grad(); loss.backward(); opt1.step()
        total_loss += loss.item()*Xb.size(0); total_n += Xb.size(0)
    print(f"[Init Epoch {epoch}] Loss={total_loss/total_n:.4f}")

# initial inference and collect self-labels
test_ds = LookaheadDataset(df, TEST_START, TEST_END, PAST_LEN, FUTURE_LEN, FEAT_COLS)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)
preds1, metas, past_windows = [], [], []
with torch.no_grad():
    for idx, (Xb, _) in enumerate(test_loader):
        Xb = Xb.to(DEVICE)
        logits = model1(Xb); batch_preds = logits.argmax(dim=1).cpu().tolist()
        for b, p in enumerate(batch_preds):
            preds1.append(p)
            metas.append(test_ds.meta[idx*BATCH_SIZE + b])
            # extract past window and duplicate
            past = Xb[b,:,:PAST_LEN].cpu().numpy()
            past_dup = np.vstack([past, past])
            past_windows.append(past_dup)

# ─── 2) SELF-LABEL RETRAIN ─────────────────────────────────────────────────
self_ds = PastSelfLabelDataset(past_windows, preds1)
self_loader = DataLoader(self_ds, batch_size=BATCH_SIZE, shuffle=True)
model2 = BiLSTM(feat_dim, HIDDEN_SIZE, NUM_LAYERS, n_labels, DROPOUT).to(DEVICE)
opt2  = torch.optim.Adam(model2.parameters(), lr=LR)
for epoch in range(1, NUM_EPOCHS+1):
    model2.train(); total_loss=0; total_n=0
    for Xb, yb in self_loader:
        Xb, yb = Xb.to(DEVICE), yb.to(DEVICE)
        logits = model2(Xb); loss = crit(logits, yb)
        opt2.zero_grad(); loss.backward(); opt2.step()
        total_loss += loss.item()*Xb.size(0); total_n += Xb.size(0)
    print(f"[Self Epoch {epoch}] Loss={total_loss/total_n:.4f}")

# ─── 3) PAST-ONLY INFERENCE & PLOTTING ───────────────────────────────────────
price_df = pd.read_csv(PRICE_CSV, sep=r"\s+", header=None)
true_cmap = ListedColormap(["#ffcccc","#f0f0f0","#ccffcc"])
pred_cmap = ListedColormap(["#ff6666","#b0b0b0","#66cc66"])
preds2 = []
with torch.no_grad():
    for pw in past_windows:
        xb = torch.from_numpy(pw).unsqueeze(0).float().to(DEVICE)
        logits = model2(xb); preds2.append(logits.argmax(dim=1).item())

from collections import defaultdict
grouped_preds2 = defaultdict(list); grouped_true = defaultdict(list); grouped_times = defaultdict(list)
for (inst,t), p2, p1 in zip(metas, preds2, preds1):
    # use original true for background
    true_lbl = int(df[(df.inst==inst)&(df.time==t)].true_regime)
    grouped_preds2[inst].append(p2)
    grouped_true[inst].append(true_lbl)
    grouped_times[inst].append(t)

# plot each instrument
for inst in sorted(grouped_preds2):
    times = np.array(grouped_times[inst])
    true_r = np.array(grouped_true[inst])
    pred_r = np.array(grouped_preds2[inst])
    price_series = price_df.iloc[:,inst]
    price_slice  = price_series.iloc[times].values
    x = np.arange(len(times))
    fig,(axT,axP) = plt.subplots(2,1, sharex=True, figsize=(12,5))
    # true
    ch = np.flatnonzero(true_r[:-1]!=true_r[1:])
    st = np.concatenate(([0], ch+1)); en = np.concatenate((ch, [len(true_r)-1]))
    for s,e,l in zip(st, en, true_r[st]): axT.axvspan(s,e, color=true_cmap(l), alpha=0.3, lw=0)
    axT.plot(x, price_slice, "k-", lw=1); axT.set_title(f"Inst {inst:02d} — TRUE")
    # pred
    ch2 = np.flatnonzero(pred_r[:-1]!=pred_r[1:])
    st2 = np.concatenate(([0], ch2+1)); en2 = np.concatenate((ch2, [len(pred_r)-1]))
    for s,e,l in zip(st2, en2, pred_r[st2]): axP.axvspan(s,e, color=pred_cmap(l), alpha=0.3, lw=0)
    axP.plot(x, price_slice, "k-", lw=1); axP.set_title(f"Inst {inst:02d} — SELF-LABEL PREDICTIONS")
    axP.set_xlabel("Test index")
    plt.tight_layout(); plt.show()


In [ ]:
import numpy  as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# ─── CONFIG ────────────────────────────────────────────────────────────────
FEATURE_CSV    = "features_labels.csv"  
PRICE_CSV      = "prices.txt"
PAST_LEN       = 20
FUTURE_LEN     = 20
TRAIN_START    = 100
TRAIN_END      = 600
TEST_START     = 600
TEST_END       = 1000
BATCH_SIZE     = 64
HIDDEN_SIZE    = 64
NUM_LAYERS     = 2
DROPOUT        = 0.2
LR             = 1e-3
NUM_EPOCHS     = 10
DEVICE         = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CONF_THRESH    = 0.90    # only accept pseudo‐labels with ≥ 90% confidence
WEIGHT_DECAY   = 1e-4    # light L2 regularization
FEAT_COLS      = ["ma","ema","slope_vol","macd","kalman","fib","psar","zscore","wret"]

# ─── DATASETS ───────────────────────────────────────────────────────────────
class LookaheadDataset(Dataset):
    def __init__(self, df, start_t, end_t, past_len, future_len, feat_cols):
        self.X, self.Y, self.meta = [], [], []
        for inst, g in df.groupby("inst"):
            g = g.sort_values("time").set_index("time")
            feats = g[feat_cols].values
            labs  = g["true_regime"].values
            times = g.index.values
            for i, t in enumerate(times):
                if t < start_t or t > end_t: 
                    continue
                if i < past_len or i + future_len >= len(times):
                    continue
                past   = feats[i-past_len  : i    ]
                future = feats[i+1          : i+1+future_len]
                window = np.vstack([past, future])
                self.X.append(window)
                self.Y.append(labs[i])
                self.meta.append((inst, t))
        self.X = torch.from_numpy(np.stack(self.X)).float()
        self.Y = torch.from_numpy(np.array(self.Y)).long()

    def __len__(self): 
        return len(self.Y)
    def __getitem__(self, i):
        return self.X[i], self.Y[i]

class PastSelfLabelDataset(Dataset):
    def __init__(self, past_windows, labels):
        self.X = torch.from_numpy(np.stack(past_windows)).float()
        self.Y = torch.from_numpy(np.array(labels)).long()

    def __len__(self): return len(self.Y)
    def __getitem__(self, i): return self.X[i], self.Y[i]

# ─── MODEL ─────────────────────────────────────────────────────────────────
class BiLSTM(nn.Module):
    def __init__(self, feat_dim, hidden, layers, n_labels, dropout):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size   = feat_dim,
            hidden_size  = hidden,
            num_layers   = layers,
            batch_first  = True,
            dropout      = dropout,
            bidirectional= True
        )
        self.fc = nn.Linear(hidden*2, n_labels)

    def forward(self, x):
        out, _ = self.lstm(x)
        out    = out[:, -1, :]
        return self.fc(out)

# ─── 1) LOAD & INITIAL TRAIN/INFER ──────────────────────────────────────────
df = pd.read_csv(FEATURE_CSV)
df = df[df.time >= TRAIN_START].reset_index(drop=True)
n_labels = int(df.true_regime.max()) + 1
feat_dim = len(FEAT_COLS)

# INITIAL TRAIN
train_ds     = LookaheadDataset(df, TRAIN_START, TRAIN_END, PAST_LEN, FUTURE_LEN, FEAT_COLS)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
model1       = BiLSTM(feat_dim, HIDDEN_SIZE, NUM_LAYERS, n_labels, DROPOUT).to(DEVICE)
opt1         = torch.optim.Adam(model1.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
crit         = nn.CrossEntropyLoss()

for epoch in range(1, NUM_EPOCHS+1):
    model1.train()
    total_loss = 0.0
    total_n    = 0
    for Xb, yb in train_loader:
        Xb, yb = Xb.to(DEVICE), yb.to(DEVICE)
        logits = model1(Xb)
        loss   = crit(logits, yb)
        opt1.zero_grad()
        loss.backward()
        opt1.step()

        total_loss += loss.item() * Xb.size(0)
        total_n    += Xb.size(0)
    print(f"[Init Epoch {epoch:02d}] Loss = {total_loss/total_n:.4f}")

# INITIAL INFERENCE + CONFIDENCE FILTERING
test_ds     = LookaheadDataset(df, TEST_START, TEST_END, PAST_LEN, FUTURE_LEN, FEAT_COLS)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)
preds1      = []
past_windows= []
meta_keep   = []

model1.eval()
with torch.no_grad():
    for idx, (Xb, _) in enumerate(test_loader):
        Xb = Xb.to(DEVICE)
        logits = model1(Xb)                     # (B, n_labels)
        probs  = torch.softmax(logits, dim=1)
        conf, batch_preds = probs.max(dim=1)    # (B,)
        for b, (c, p) in enumerate(zip(conf.cpu().tolist(),
                                       batch_preds.cpu().tolist())):
            if c < CONF_THRESH:
                continue
            idx_global = idx * BATCH_SIZE + b
            preds1.append(p)
            meta_keep.append(test_ds.meta[idx_global])
            # only keep past for the student
            past = Xb[b,:,:PAST_LEN].cpu().numpy()
            past_dup = np.vstack([past, past])
            past_windows.append(past_dup)

# ─── 2) SELF-LABEL RETRAIN ─────────────────────────────────────────────────
self_ds     = PastSelfLabelDataset(past_windows, preds1)
self_loader = DataLoader(self_ds, batch_size=BATCH_SIZE, shuffle=True)
model2      = BiLSTM(feat_dim, HIDDEN_SIZE, NUM_LAYERS, n_labels, DROPOUT).to(DEVICE)
opt2        = torch.optim.Adam(model2.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

for epoch in range(1, NUM_EPOCHS+1):
    model2.train()
    total_loss = 0.0
    total_n    = 0
    for Xb, yb in self_loader:
        Xb, yb = Xb.to(DEVICE), yb.to(DEVICE)
        logits = model2(Xb)
        loss   = crit(logits, yb)
        opt2.zero_grad()
        loss.backward()
        opt2.step()

        total_loss += loss.item() * Xb.size(0)
        total_n    += Xb.size(0)
    print(f"[Self  Epoch {epoch:02d}] Loss = {total_loss/total_n:.4f}")

# ─── 3) PAST‐ONLY INFERENCE & PLOTTING ───────────────────────────────────────
price_df   = pd.read_csv(PRICE_CSV, sep=r"\s+", header=None)
true_cmap  = ListedColormap(["#ffcccc","#f0f0f0","#ccffcc"])
pred_cmap  = ListedColormap(["#ff6666","#b0b0b0","#66cc66"])

# collect model2 predictions
preds2 = []
model2.eval()
with torch.no_grad():
    for pw in past_windows:
        xb = torch.from_numpy(pw).unsqueeze(0).float().to(DEVICE)
        logits = model2(xb)
        preds2.append(logits.argmax(dim=1).item())

from collections import defaultdict
grouped_preds2 = defaultdict(list)
grouped_true   = defaultdict(list)
grouped_times  = defaultdict(list)

for (inst,t), p2 in zip(meta_keep, preds2):
    true_lbl = int(df[(df.inst==inst)&(df.time==t)].true_regime)
    grouped_preds2[inst].append(p2)
    grouped_true[inst].append(true_lbl)
    grouped_times[inst].append(t)

def get_segments(r):
    ch = np.flatnonzero(r[:-1]!=r[1:])
    st = np.concatenate(([0], ch+1))
    en = np.concatenate((ch, [len(r)-1]))
    return list(zip(st,en,r[st]))

green = "#ccffcc"; red = "#ffcccc"

for inst in sorted(grouped_preds2):
    times     = np.array(grouped_times[inst])
    true_r    = np.array(grouped_true[inst])
    pred_r    = np.array(grouped_preds2[inst])
    price     = price_df.iloc[times, inst].values
    x         = np.arange(len(times))

    fig, (axT, axP) = plt.subplots(2,1, sharex=True, figsize=(12,5))
    # TRUE
    for s,e,l in get_segments(true_r):
        color = green if l==2 else red if l==0 else "lightgrey"
        axT.axvspan(x[s], x[e], color=color, alpha=0.3, lw=0)
    axT.plot(x, price, "k-", lw=1)
    axT.set_title(f"Inst {inst:02d} — TRUE regimes")
    # PREDICTED
    for s,e,l in get_segments(pred_r):
        color = green if l==2 else red if l==0 else "lightgrey"
        axP.axvspan(x[s], x[e], color=color, alpha=0.3, lw=0)
    axP.plot(x, price, "k-", lw=1)
    axP.set_title(f"Inst {inst:02d} — SELF-LABEL PREDICTIONS (conf≥{CONF_THRESH})")
    axP.set_xlabel("Time Index")
    plt.tight_layout()
    plt.show()


In [ ]:
import numpy  as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# ─── CONFIG ────────────────────────────────────────────────────────────────
FEATURE_CSV    = "features_labels.csv"  
PRICE_CSV      = "prices.txt"
PAST_LEN       = 20
FUTURE_LEN     = 20
TRAIN_START    = 100
TRAIN_END      = 600
TEST_START     = 600
TEST_END       = 1000
BATCH_SIZE     = 64
HIDDEN_SIZE    = 64
NUM_LAYERS     = 2
DROPOUT        = 0.2
LR             = 1e-3
NUM_EPOCHS     = 10
DEVICE         = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CONF_THRESH    = 0.70    # only accept pseudo‐labels with ≥ 90% confidence
WEIGHT_DECAY   = 1e-4    # light L2 regularization
FEAT_COLS      = ["ma","ema","slope_vol","macd","kalman","fib","psar","zscore","wret"]

# ─── DATASETS ───────────────────────────────────────────────────────────────
class LookaheadDataset(Dataset):
    def __init__(self, df, start_t, end_t, past_len, future_len, feat_cols):
        self.X, self.Y, self.meta = [], [], []
        for inst, g in df.groupby("inst"):
            g = g.sort_values("time").set_index("time")
            feats = g[feat_cols].values
            labs  = g["true_regime"].values
            times = g.index.values
            for i, t in enumerate(times):
                if t < start_t or t > end_t: 
                    continue
                if i < past_len or i + future_len >= len(times):
                    continue
                past   = feats[i-past_len  : i    ]
                future = feats[i+1          : i+1+future_len]
                window = np.vstack([past, future])
                self.X.append(window)
                self.Y.append(labs[i])
                self.meta.append((inst, t))
        self.X = torch.from_numpy(np.stack(self.X)).float()
        self.Y = torch.from_numpy(np.array(self.Y)).long()

    def __len__(self): 
        return len(self.Y)
    def __getitem__(self, i):
        return self.X[i], self.Y[i]

class PastSelfLabelDataset(Dataset):
    def __init__(self, past_windows, labels):
        self.X = torch.from_numpy(np.stack(past_windows)).float()
        self.Y = torch.from_numpy(np.array(labels)).long()

    def __len__(self): return len(self.Y)
    def __getitem__(self, i): return self.X[i], self.Y[i]

# ─── MODEL ─────────────────────────────────────────────────────────────────
class BiLSTM(nn.Module):
    def __init__(self, feat_dim, hidden, layers, n_labels, dropout):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size   = feat_dim,
            hidden_size  = hidden,
            num_layers   = layers,
            batch_first  = True,
            dropout      = dropout,
            bidirectional= True
        )
        self.fc = nn.Linear(hidden*2, n_labels)

    def forward(self, x):
        out, _ = self.lstm(x)
        out    = out[:, -1, :]
        return self.fc(out)

# ─── 1) LOAD & INITIAL TRAIN/INFER ──────────────────────────────────────────
df = pd.read_csv(FEATURE_CSV)
df = df[df.time >= TRAIN_START].reset_index(drop=True)
n_labels = int(df.true_regime.max()) + 1
feat_dim = len(FEAT_COLS)

# INITIAL TRAIN
train_ds     = LookaheadDataset(df, TRAIN_START, TRAIN_END, PAST_LEN, FUTURE_LEN, FEAT_COLS)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
model1       = BiLSTM(feat_dim, HIDDEN_SIZE, NUM_LAYERS, n_labels, DROPOUT).to(DEVICE)
opt1         = torch.optim.Adam(model1.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
crit         = nn.CrossEntropyLoss()

for epoch in range(1, NUM_EPOCHS+1):
    model1.train()
    total_loss = 0.0
    total_n    = 0
    for Xb, yb in train_loader:
        Xb, yb = Xb.to(DEVICE), yb.to(DEVICE)
        logits = model1(Xb)
        loss   = crit(logits, yb)
        opt1.zero_grad()
        loss.backward()
        opt1.step()

        total_loss += loss.item() * Xb.size(0)
        total_n    += Xb.size(0)
    print(f"[Init Epoch {epoch:02d}] Loss = {total_loss/total_n:.4f}")

# INITIAL INFERENCE + CONFIDENCE FILTERING
test_ds     = LookaheadDataset(df, TEST_START, TEST_END, PAST_LEN, FUTURE_LEN, FEAT_COLS)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)
preds1      = []
past_windows= []
meta_keep   = []

model1.eval()
with torch.no_grad():
    for idx, (Xb, _) in enumerate(test_loader):
        Xb = Xb.to(DEVICE)
        logits = model1(Xb)                     # (B, n_labels)
        probs  = torch.softmax(logits, dim=1)
        conf, batch_preds = probs.max(dim=1)    # (B,)
        for b, (c, p) in enumerate(zip(conf.cpu().tolist(),
                                       batch_preds.cpu().tolist())):
            if c < CONF_THRESH:
                continue
            idx_global = idx * BATCH_SIZE + b
            preds1.append(p)
            meta_keep.append(test_ds.meta[idx_global])
            # only keep past for the student
            past = Xb[b,:,:PAST_LEN].cpu().numpy()
            past_dup = np.vstack([past, past])
            past_windows.append(past_dup)

# ─── 2) SELF-LABEL RETRAIN ─────────────────────────────────────────────────
self_ds     = PastSelfLabelDataset(past_windows, preds1)
self_loader = DataLoader(self_ds, batch_size=BATCH_SIZE, shuffle=True)
model2      = BiLSTM(feat_dim, HIDDEN_SIZE, NUM_LAYERS, n_labels, DROPOUT).to(DEVICE)
opt2        = torch.optim.Adam(model2.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

for epoch in range(1, NUM_EPOCHS+1):
    model2.train()
    total_loss = 0.0
    total_n    = 0
    for Xb, yb in self_loader:
        Xb, yb = Xb.to(DEVICE), yb.to(DEVICE)
        logits = model2(Xb)
        loss   = crit(logits, yb)
        opt2.zero_grad()
        loss.backward()
        opt2.step()

        total_loss += loss.item() * Xb.size(0)
        total_n    += Xb.size(0)
    print(f"[Self  Epoch {epoch:02d}] Loss = {total_loss/total_n:.4f}")

# ─── 3) PAST‐ONLY INFERENCE & PLOTTING ───────────────────────────────────────
price_df   = pd.read_csv(PRICE_CSV, sep=r"\s+", header=None)
true_cmap  = ListedColormap(["#ffcccc","#f0f0f0","#ccffcc"])
pred_cmap  = ListedColormap(["#ff6666","#b0b0b0","#66cc66"])

# collect model2 predictions
preds2 = []
model2.eval()
with torch.no_grad():
    for pw in past_windows:
        xb = torch.from_numpy(pw).unsqueeze(0).float().to(DEVICE)
        logits = model2(xb)
        preds2.append(logits.argmax(dim=1).item())

from collections import defaultdict
grouped_preds2 = defaultdict(list)
grouped_true   = defaultdict(list)
grouped_times  = defaultdict(list)

for (inst,t), p2 in zip(meta_keep, preds2):
    true_lbl = int(df[(df.inst==inst)&(df.time==t)].true_regime)
    grouped_preds2[inst].append(p2)
    grouped_true[inst].append(true_lbl)
    grouped_times[inst].append(t)

def get_segments(r):
    ch = np.flatnonzero(r[:-1]!=r[1:])
    st = np.concatenate(([0], ch+1))
    en = np.concatenate((ch, [len(r)-1]))
    return list(zip(st,en,r[st]))

green = "#ccffcc"; red = "#ffcccc"

for inst in sorted(grouped_preds2):
    times     = np.array(grouped_times[inst])
    true_r    = np.array(grouped_true[inst])
    pred_r    = np.array(grouped_preds2[inst])
    price     = price_df.iloc[times, inst].values
    x         = np.arange(len(times))

    fig, (axT, axP) = plt.subplots(2,1, sharex=True, figsize=(12,5))
    # TRUE
    for s,e,l in get_segments(true_r):
        color = green if l==2 else red if l==0 else "lightgrey"
        axT.axvspan(x[s], x[e], color=color, alpha=0.3, lw=0)
    axT.plot(x, price, "k-", lw=1)
    axT.set_title(f"Inst {inst:02d} — TRUE regimes")
    # PREDICTED
    for s,e,l in get_segments(pred_r):
        color = green if l==2 else red if l==0 else "lightgrey"
        axP.axvspan(x[s], x[e], color=color, alpha=0.3, lw=0)
    axP.plot(x, price, "k-", lw=1)
    axP.set_title(f"Inst {inst:02d} — SELF-LABEL PREDICTIONS (conf≥{CONF_THRESH})")
    axP.set_xlabel("Time Index")
    plt.tight_layout()
    plt.show()
